In [1]:
from sample_bot import *
import shutil
import argparse
from datetime import datetime

import socketio
import eventlet
import eventlet.wsgi
from flask import Flask

import math
from queue import Queue

import sys
sys.path.append('/home/louner/school/ml/reinforcement-learning/')
from lab import *

import tensorflow.contrib.slim as slim
from random import random, choice

from time import sleep
import pyautogui

import traceback

In [2]:
batch_size = 100
max_size = batch_size*10
t_max = 5
reload_window = 100
reward_decay = 0.99
last_state = None
last_action = None
last_reward = None
learning_rate = 0.001
mu = 0.999

In [3]:
# 11 possible steering angles, 0 and 5 uniformly selected angle from 0~pi and 0~-1*pi
#ANGLES = [-3.141592653589793, -2.5132741228718345, -1.8849555921538759, -1.2566370614359172, -0.6283185307179586, 0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793]
ANGLES = [-0.349065850398866, -0.3316125578789227, -0.31415926535897937, -0.29670597283903605, -0.2792526803190927, -0.2617993877991494, -0.24434609527920612, -0.22689280275926282, -0.20943951023931953, -0.19198621771937624, -0.17453292519943295, -0.15707963267948966, -0.13962634015954636, -0.12217304763960306, -0.10471975511965977, -0.08726646259971647, -0.06981317007977318, -0.05235987755982989, -0.03490658503988659, -0.017453292519943295, 0, 0.017453292519943295, 0.03490658503988659, 0.05235987755982989, 0.06981317007977318, 0.08726646259971647, 0.10471975511965977, 0.12217304763960306, 0.13962634015954636, 0.15707963267948966, 0.17453292519943295, 0.19198621771937624, 0.20943951023931953, 0.22689280275926282, 0.24434609527920612, 0.2617993877991494, 0.2792526803190927, 0.29670597283903605, 0.31415926535897937, 0.3316125578789227, 0.349065850398866]
#ANGLES = [0.0]
# 3 possible throttle options
THROTTLES = [0.0, 0.5, 1.0]
#THROTTLES = [0.0, 0.1]
ACTIONS = [(angle, throttle) for angle in ANGLES for throttle in THROTTLES] #33 actions

In [4]:
def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': str(steering_angle),
            'throttle': str(throttle)
        },  
        skip_sid=True)
    
def calculate_reward(steering_angle, speed):
    return speed * math.cos(steering_angle)

class Action:
    def __init__(self, action_idx):
        self.action_idx = action_idx
        self.steering_angle = ACTIONS[action_idx][0]
        self.throttle = ACTIONS[action_idx][1]
        
    def __repr__(self):
        return str({'throttle': self.throttle, 'angle': self.steering_angle})

In [5]:
class Transaction:
    columns = ['state', 'action', 'reward', 'next_state', 'advantage']
    
    def __init__(self, item):
        self.item = dict(zip(Transaction.columns, item))
    
    def  __lt__(self, other):
        #sort by time
        return self.item['id'] < other.item['id']
        #sort by gained label
        #return self.item['label'] < other.item['label']
    
    def __repr__(self):
        return str(self.item)
        
    def __getitem__(self, key):
        return self.item[key]
    
    def __setitem__(self, key, value):
        self.item[key] = value


In [6]:
# policy & value network only different at last layer
# common part is cnn1-cnn1-fc
def build_network():
    with tf.variable_scope('Action-Critic-Network-Common', reuse=tf.AUTO_REUSE):
        #inputs = tf.placeholder(shape=(None, 108, 320, 3), dtype=tf.float32) # track image
        inputs = tf.placeholder(shape=(None, 240, 320, 3), dtype=tf.float32) # track image
        layer_1 = slim.conv2d(inputs=inputs, num_outputs=16, kernel_size=(8, 8), stride=(8, 8), padding='VALID', activation_fn=tf.nn.elu)
        layer_2 = slim.conv2d(inputs=layer_1, num_outputs=32, kernel_size=(4, 4), stride=(4, 4), padding='VALID', activation_fn=tf.nn.elu)
        fc_layer = tf.layers.dense(inputs=slim.flatten(layer_2), units=256, activation=tf.nn.relu)
    return inputs, fc_layer

In [7]:
# P(state) = action probability
class PolicyNetwork:
    def __init__(self):
        self.action_size = len(ACTIONS)
        
        self.advantage = tf.placeholder(shape=(None, self.action_size), dtype=tf.float32)
        
        inputs, output = build_network()
        self.inputs = inputs
    
        self.metrics = {}
        self.metrics['loss'] = []
        
        with tf.variable_scope('policy-network'):
            self.actions_values = tf.layers.dense(output, units=self.action_size, activation=tf.nn.softmax) # batch_size, action_size
            self.best_action = tf.argmax(self.actions_values, axis=1)

            self.loss = tf.reduce_sum(tf.log(self.actions_values) * self.advantage)*-1
            self.train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)

        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
    
    def get_action(self, state):
        best_action = self.sess.run(self.best_action, feed_dict={self.inputs: [state]})[0]
        return best_action
    
    def update(self, batch):
        states = [transaction['state'] for transaction in batch]
        advantages = []
        for transaction in batch:
            # unselected actions get 0 advantage, selected action get reward - current_state_value advantage
            advantage = [0]*self.action_size
            advantage[transaction['action']] = transaction['advantage']

            advantages.append(advantage)
            
        advantages = np.array(advantages)
        
        _, loss = self.sess.run([self.train_op, self.loss], feed_dict={self.inputs: states, self.advantage: advantages})
        self.metrics['loss'].append(loss)

In [8]:
# V(state) = state_value
class ValueNetwork:
    def __init__(self):
        self.output_size = 1
        self.model_folder = './model/value_network'
        
        self.rewards = tf.placeholder(shape=(None, 1), dtype=tf.float32)
        inputs, output = build_network()
        
        with tf.variable_scope('value-network'):
            self.inputs = inputs
            self.state_value = tf.layers.dense(output, units=self.output_size)

            self.loss = tf.reduce_sum(tf.square(self.state_value - self.rewards))
            self.train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)

        self.sess = tf.Session()
        self.target_sess = tf.Session()
        
        self.sess.run(tf.global_variables_initializer())
        self.target_sess.run(tf.global_variables_initializer())

        self.saver = tf.train.Saver()
        
        self.metrics = {}
        self.metrics['loss'] = []
    
    def get_state_values(self, states):
        state_values = self.target_sess.run(self.state_value, feed_dict={self.inputs: states})
        return state_values
    
    def get_state_value(self, state):
        return self.get_state_values([state])[0]
    
    def update(self, batch):
        inputs = [transaction['state'] for transaction in batch]
        rewards = [transaction['reward'] for transaction in batch]
        
        _, loss = self.sess.run([self.train_op, self.loss], feed_dict={self.inputs: inputs, self.rewards: rewards})
        self.metrics['loss'].append(loss)
    
    def reload(self, model_name):
        self.saver.save(self.sess, '%s/%s'%(self.model_folder, model_name))
        self.saver.restore(self.target_sess, tf.train.latest_checkpoint(self.model_folder))

In [9]:
# when t_max transactions are collected, calculate first transaction's reward/advantage
def add_advantage(transaction, transactions, value_network):
    final_state = transactions[-1]['state']
    final_state_value = value_network.get_state_value(final_state)
    
    reward = final_state_value
    for t in reversed(transactions):
        reward += t['reward']
        reward *= reward_decay
        
    transaction['reward'] = reward
    
    current_state = transaction['state']
    current_state_value = value_network.get_state_value(current_state)
    
    advantage = reward - current_state_value
    transaction['advantage'] = advantage

In [13]:
class ACNDrive(AutoDrive):
    def __init__(self, *args, **kwargs):
        AutoDrive.__init__(self, *args, **kwargs)
        
        self.policy_network = PolicyNetwork()
        self.value_network = ValueNetwork()
        
        self.memory = Memory(max_size=max_size)
        self.transactions = []
        self.round_id = 1
        
        self.last_state = None
        self.last_action = None
        self.last_reward = None
        
        self.mu = 0.5
        self.speeds = [1]*10
        
    def update_speeds(self, speed):
        self.speeds = self.speeds[1:]
        self.speeds.append(speed)

    def is_crash(self):
        return np.mean(self.speeds) <= 0.01

    def end_and_restart_new_game(self):
        print('crashed, restarting...')
        pyautogui.press('esc')
        # my mouse ...
        for _ in range(10):
            pyautogui.click()
        
        self.speeds = [1]*10

    def on_dashboard(self, src_img, last_steering_angle, speed, throttle, info):        
        # get angle to calculate reward
        self.update_speeds(speed)
        if self.is_crash():
            self.end_and_restart_new_game()

        try:
            track_img     = ImageProcessor.preprocess(src_img)
            current_angle = ImageProcessor.find_steering_angle_by_color(track_img, last_steering_angle, debug = self.debug)
            steering_angle = self._steering_pid.update(-current_angle)
            throttle       = self._throttle_pid.update(speed)
            
        except:
            print(traceback.format_exc())
            print(self.speeds)
            self._car.control(0, 0)
            return
                            
        if True:
            ImageProcessor.show_image(src_img, "source")
            ImageProcessor.show_image(track_img, "track")
            logit("steering PID: %0.2f (%0.2f) => %0.2f (%0.2f)" % (current_angle, ImageProcessor.rad2deg(current_angle), steering_angle, ImageProcessor.rad2deg(steering_angle)))
            logit("throttle PID: %0.4f => %0.4f" % (speed, throttle))
            logit("info: %s" % repr(info))
        
        # update last transaction
        # because can't get next state right after executing action QQ
        state = src_img
        if self.last_state is not None:
            transaction = Transaction([self.last_state, self.last_action, self.last_reward, state, None])
            self.transactions.append(transaction)
        
        # select action according to current state
        action = self.policy_network.get_action(state)
        
        if random() < self.mu:
            action = randint(a=0, b=len(ACTIONS)-1)
        action = Action(action)
        steering_angle, throttle = action.steering_angle, action.throttle
        
        self.last_action = action.action_idx
        self.last_reward = calculate_reward(steering_angle, speed)
        self.last_state = state
        
        # execute action
        self._car.control(steering_angle, throttle)
        self.round_id += 1
        
        print(len(drive.memory.d), len(self.transactions), self.last_reward, steering_angle, self.speeds)
        
        # update policy & value network
        batch = self.memory.batch(batch_size)
        if batch.any():
            self.policy_network.update(batch)
            self.value_network.update(batch)
        
        # when storing t_max transactions, update first transaction's label, pop & push it into memory
        if len(self.transactions) == t_max:
            transaction = self.transactions[0]
            add_advantage(transaction, self.transactions, self.value_network)
            self.memory.insert(transaction)
            self.transactions = self.transactions[1:]
        
        # reload policy & value network periodically
        if self.round_id % reload_window == 0:
            #policy_network.reload() #policy network need reload ?
            self.value_network.reload(str(info))
            
        #self.mu *= self.mu

In [14]:
tf.reset_default_graph()

sio = socketio.Server()
record_folder = './records/'
car = Car(control_function = send_control)
#drive = AutoDrive(car, record_folder=record_folder)
drive = ACNDrive(car, record_folder=record_folder)

@sio.on('telemetry')
def telemetry(sid, dashboard):
    if dashboard:
        car.on_dashboard(dashboard)
    else:
        sio.emit('manual', data={}, skip_sid=True)

@sio.on('connect')
def connect(sid, environ):
    car.control(0, 0)

In [15]:
app = socketio.Middleware(sio, Flask(__name__))
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(20982) wsgi starting up on http://0.0.0.0:4567
(20982) accepted ('127.0.0.1', 58650)


0 0 0.40594290587773424 0.05235987755982989 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0.4065]
0 1 0.40594290587773424 -0.05235987755982989 [1, 1, 1, 1, 1, 1, 1, 1, 0.4065, 0.4065]
0 2 0.4062523711832624 0.03490658503988659 [1, 1, 1, 1, 1, 1, 1, 0.4065, 0.4065, 0.4065]
0 3 0.011248697408438766 -0.20943951023931953 [1, 1, 1, 1, 1, 1, 0.4065, 0.4065, 0.4065, 0.0115]
0 4 0.011205255745030205 0.22689280275926282 [1, 1, 1, 1, 1, 0.4065, 0.4065, 0.4065, 0.0115, 0.0115]
0 5 0.010806465139037946 -0.349065850398866 [1, 1, 1, 1, 0.4065, 0.4065, 0.4065, 0.0115, 0.0115, 0.0115]
1 5 0.08756114476353163 0.19198621771937624 [1, 1, 1, 0.4065, 0.4065, 0.4065, 0.0115, 0.0115, 0.0115, 0.0892]
2 5 0.14292724762153666 -0.349065850398866 [1, 1, 0.4065, 0.4065, 0.4065, 0.0115, 0.0115, 0.0115, 0.0892, 0.1521]
3 5 0.14292724762153666 -0.349065850398866 [1, 0.4065, 0.4065, 0.4065, 0.0115, 0.0115, 0.0115, 0.0892, 0.1521, 0.1521]
4 5 0.2872640341742522 -0.349065850398866 [0.4065, 0.4065, 0.4065, 0.0115, 0.0115, 0.0115, 0.0892, 

/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.4/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Traceback (most recent call last):
  File "<ipython-input-13-863977505b8b>", line 42, in on_dashboard
    current_angle = ImageProcessor.find_steering_angle_by_color(track_img, last_steering_angle, debug = self.debug)
  File "/home/louner/school/ml/formula-trend-1.0.0-alpha.4/sample_bot_v1.0.2/sample_bot.py", line 372, in find_steering_angle_by_color
    x = int(image_width / 2 + int(r * math.cos(steering_angle)))
ValueError: cannot convert float NaN to integer

[0.7775, 0.7775, 0.7775, 0.8824, 0.8824, 0.8824, 0.8541, 0.8541, 0.8541, 0.9204]
Traceback (most recent call last):
  File "<ipython-input-13-863977505b8b>", line 42, in on_dashboard
    current_angle = ImageProcessor.find_steering_angle_by_color(track_img, last_steering_angle, debug = self.debug)
  File "/home/louner/school/ml/formula-trend-1.0.0-alpha.4/sample_bot_v1.0.2/sample_bot.py", line 372, in find_steering_angle_by_color
    x = int(image_width / 2 + int(r * math.cos(steering_angle)))
ValueError: cannot convert float N

(20982) accepted ('127.0.0.1', 34496)


...

127.0.0.1 - - [05/Sep/2018 09:40:17] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 62.752936


982 5 0.3841107168830709 -0.19198621771937624 [1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3913]
983 5 0.42124907726382627 -0.15707963267948966 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3913, 0.4265]
984 5 0.42124907726382627 0.15707963267948966 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3913, 0.4265, 0.4265]
985 5 0.6156866051389271 -0.349065850398866 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3913, 0.4265, 0.4265, 0.6552]
986 5 0.6740415168897321 -0.349065850398866 [0.0, 0.0, 0.0, 0.0, 0.0, 0.3913, 0.4265, 0.4265, 0.6552, 0.7173]
987 5 0.7171907517356795 0.017453292519943295 [0.0, 0.0, 0.0, 0.0, 0.3913, 0.4265, 0.4265, 0.6552, 0.7173, 0.7173]
988 5 0.8840412613402562 0.06981317007977318 [0.0, 0.0, 0.0, 0.3913, 0.4265, 0.4265, 0.6552, 0.7173, 0.7173, 0.8862]
989 5 0.7891538629360058 -0.349065850398866 [0.0, 0.0, 0.3913, 0.4265, 0.4265, 0.6552, 0.7173, 0.7173, 0.8862, 0.8398]
990 5 0.7891538629360058 -0.349065850398866 [0.0, 0.3913, 0.4265, 0.4265, 0.6552, 0.7173, 0.7173, 0.8862, 0.8398, 0.8398]
99

127.0.0.1 - - [05/Sep/2018 09:40:19] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 62.313281


Traceback (most recent call last):
  File "<ipython-input-13-863977505b8b>", line 42, in on_dashboard
    current_angle = ImageProcessor.find_steering_angle_by_color(track_img, last_steering_angle, debug = self.debug)
  File "/home/louner/school/ml/formula-trend-1.0.0-alpha.4/sample_bot_v1.0.2/sample_bot.py", line 372, in find_steering_angle_by_color
    x = int(image_width / 2 + int(r * math.cos(steering_angle)))
ValueError: cannot convert float NaN to integer

[0.1538, 0.1538, 0.1538, 0.1368, 0.1368, 0.1246, 0.1246, 0.1133, 0.1133, 0.1038]
Traceback (most recent call last):
  File "<ipython-input-13-863977505b8b>", line 42, in on_dashboard
    current_angle = ImageProcessor.find_steering_angle_by_color(track_img, last_steering_angle, debug = self.debug)
  File "/home/louner/school/ml/formula-trend-1.0.0-alpha.4/sample_bot_v1.0.2/sample_bot.py", line 372, in find_steering_angle_by_color
    x = int(image_width / 2 + int(r * math.cos(steering_angle)))
ValueError: cannot convert float N

127.0.0.1 - - [05/Sep/2018 09:40:21] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 62.307379
127.0.0.1 - - [05/Sep/2018 09:40:23] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 29.427376
127.0.0.1 - - [05/Sep/2018 09:40:23] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 41.828928
127.0.0.1 - - [05/Sep/2018 09:40:23] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 9.264388
127.0.0.1 - - [05/Sep/2018 09:40:23] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 47.973671
127.0.0.1 - - [05/Sep/2018 09:40:23] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 53.349123
127.0.0.1 - - [05/Sep/2018 09:40:23] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 24.952232
127.0.0.1 - - [05/Sep/2018 09:40:23] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 55.722312
127.0.0.1 - - [05/Sep/2018 09:40:23] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 31.711198
127.0.0.1 - - [05/Sep/2018 09:40:23] "GET /sock

In [ ]:
np.mean( [0.0015, 0.0002, 0.0004, 0.0004, 0.0003, 0.0003, 0.0003, 0.0002, 0.0002, 0.0002])

0.0004

In [ ]:
%%bash
cd ../Linux/
sh StartGame_local.sh &

In [17]:
largest_angle = math.pi/9
angle, step_angle = 0, largest_angle/20.0
angles = [angle]

while angle < largest_angle:
    angle += step_angle
    angles.append(angle)
    angles.append(angle*-1)
print(sorted(angles))

[-0.349065850398866, -0.3316125578789227, -0.31415926535897937, -0.29670597283903605, -0.2792526803190927, -0.2617993877991494, -0.24434609527920612, -0.22689280275926282, -0.20943951023931953, -0.19198621771937624, -0.17453292519943295, -0.15707963267948966, -0.13962634015954636, -0.12217304763960306, -0.10471975511965977, -0.08726646259971647, -0.06981317007977318, -0.05235987755982989, -0.03490658503988659, -0.017453292519943295, 0, 0.017453292519943295, 0.03490658503988659, 0.05235987755982989, 0.06981317007977318, 0.08726646259971647, 0.10471975511965977, 0.12217304763960306, 0.13962634015954636, 0.15707963267948966, 0.17453292519943295, 0.19198621771937624, 0.20943951023931953, 0.22689280275926282, 0.24434609527920612, 0.2617993877991494, 0.2792526803190927, 0.29670597283903605, 0.31415926535897937, 0.3316125578789227, 0.349065850398866]


In [15]:
import pyautogui
for i in range(10):
    pyautogui.press('esc')
    #pyautogui.click()
    print(i)
    sleep(1)
    
    

0
1
2
3
4
5


KeyboardInterrupt: 

In [6]:
pyautogui.KEYBOARD_KEYS




['\t',
 '\n',
 '\r',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 'accept',
 'add',
 'alt',
 'altleft',
 'altright',
 'apps',
 'backspace',
 'browserback',
 'browserfavorites',
 'browserforward',
 'browserhome',
 'browserrefresh',
 'browsersearch',
 'browserstop',
 'capslock',
 'clear',
 'convert',
 'ctrl',
 'ctrlleft',
 'ctrlright',
 'decimal',
 'del',
 'delete',
 'divide',
 'down',
 'end',
 'enter',
 'esc',
 'escape',
 'execute',
 'f1',
 'f10',
 'f11',
 'f12',
 'f13',
 'f14',
 'f15',
 'f16',
 'f17',
 'f18',
 'f19',
 'f2',
 'f20',
 'f21',
 'f22',
 'f23',
 'f24',
 'f3',
 'f4',
 'f5',
 'f6',
 'f7',
 'f8',
 'f9',
 'final',
